# **Face Detection**
---

## Face Crop the downloaded images

In [ ]:
import os
import cv2
import time
import numpy as np

classifier = cv2.CascadeClassifier(r"haarcascade_frontalface_default.xml")

In [40]:
data = []

In [41]:
for img in os.listdir(r'downloaded_images'):    
    frame = cv2.imread(os.path.join("downloaded_images",img))
    
    face_points = classifier.detectMultiScale(frame,1.3,5)
    
    if len(face_points)>0:
        for x,y,w,h in face_points:
            face_frame = frame[y:y+h+1,x:x+w+1]
            cv2.imshow("Only face",face_frame)
            if len(data)<=100:
                data.append(face_frame)
                break
    # time.sleep(1)
    if cv2.waitKey(30) == ord("q"):
        break
cv2.destroyAllWindows()

In [42]:
name = input("Enter Face holder name : ")
for i in range(len(data)):
    cv2.imwrite("images/"+name+"_"+str(i)+".jpg",data[i])
print("Done")

Done


## Pickle the images

In [3]:
import pickle

data_dir = os.path.join(os.getcwd(),'data_dir')
img_dir = os.path.join(os.getcwd(),'output')

image_data = []
labels = []

for i in os.listdir(img_dir):
    image = cv2.imread(os.path.join(img_dir,i))
    image = cv2.resize(image,(100,100))
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image_data.append(image)
    labels.append(str(i).split("_")[0])
    
image_data = np.array(image_data)    
labels = np.array(labels) 

In [4]:
with open(os.path.join(data_dir,"images.p"),'wb') as f:
    pickle.dump(image_data,f)
    
with open(os.path.join(data_dir,"labels.p"),'wb') as f:
    pickle.dump(labels,f)

## Load the pickle

In [15]:
import pickle
import numpy as np

with open(r'data_dir/images.p', 'rb') as f:
    images = pickle.load(f)

with open(r'data_dir/labels.p', 'rb') as f:
    labels = pickle.load(f)

In [16]:
print(images.shape)
print(labels.shape)

(390, 100, 100)
(390,)


In [17]:
set(labels)

{'alia', 'deepika', 'jennie', 'srk', 'v'}

In [18]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(labels)
print(labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


In [19]:
set(labels)

{0, 1, 2, 3, 4}

In [20]:
le.inverse_transform([0,1,2,3,4])

array(['alia', 'deepika', 'jennie', 'srk', 'v'], dtype='<U7')

In [21]:
n_persons = len(set(labels))

In [22]:
l = le.inverse_transform(np.arange(n_persons))
for i in range(len(l)):
    print(i, "--->", l[i])

0 ---> alia
1 ---> deepika
2 ---> jennie
3 ---> srk
4 ---> v


## Model Training

In [23]:
import cv2

def preprocessing(img):
    img = cv2.equalizeHist(img)
    img = img.reshape(100, 100, 1)
    img = img/255
    return img

In [24]:
images = np.array(list(map(preprocessing, images)))
print("Shape of Input image: ", images.shape)

Shape of Input image:  (390, 100, 100, 1)


In [25]:
from keras.utils import to_categorical
labels = to_categorical(labels)

In [26]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [83]:
def Lenet_Model():
    model = Sequential()
    model.add( Conv2D(32, (5,5), input_shape=(100,100,1), activation='relu') )
    model.add( MaxPooling2D(pool_size = (2,2)) )

    model.add( Conv2D(64, (3,3), activation='relu') )
    model.add( MaxPooling2D(pool_size = (2,2)) )


    model.add( Flatten() )

    model.add( Dense(128, activation='relu') )
    model.add( Dense(64, activation='relu') )
    model.add( Dense(128, activation='relu') )

    model.add( Dense(5, activation='softmax') )
    model.compile(Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [89]:
model = Lenet_Model()
model.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_58 (Conv2D)          (None, 96, 96, 32)        832       
                                                                 
 max_pooling2d_58 (MaxPooli  (None, 48, 48, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_59 (Conv2D)          (None, 46, 46, 64)        18496     
                                                                 
 max_pooling2d_59 (MaxPooli  (None, 23, 23, 64)        0         
 ng2D)                                                           
                                                                 
 flatten_24 (Flatten)        (None, 33856)             0         
                                                                 
 dense_81 (Dense)            (None, 128)             

In [254]:
h = model.fit(images, labels, validation_split=0.1, epochs=15)

Epoch 1/15
11/11 [==============================] - 6s 259ms/step - loss: 2.3399 - accuracy: 0.3105 - val_loss: 1.6200 - val_accuracy: 0.0000e+00
Epoch 2/15
11/11 [==============================] - 2s 225ms/step - loss: 2.0071 - accuracy: 0.3675 - val_loss: 1.3865 - val_accuracy: 0.5385
Epoch 3/15
11/11 [==============================] - 2s 212ms/step - loss: 1.5443 - accuracy: 0.4587 - val_loss: 0.9466 - val_accuracy: 1.0000
Epoch 4/15
11/11 [==============================] - 2s 215ms/step - loss: 1.2523 - accuracy: 0.5926 - val_loss: 0.7281 - val_accuracy: 1.0000
Epoch 5/15
11/11 [==============================] - 2s 219ms/step - loss: 1.0561 - accuracy: 0.5954 - val_loss: 0.5484 - val_accuracy: 1.0000
Epoch 6/15
11/11 [==============================] - 2s 217ms/step - loss: 1.0442 - accuracy: 0.6610 - val_loss: 0.3469 - val_accuracy: 1.0000
Epoch 7/15
11/11 [==============================] - 2s 208ms/step - loss: 0.8583 - accuracy: 0.7123 - val_loss: 0.2428 - val_accuracy: 1.0000
Ep

In [197]:
model.save('final_model.h5')

c:\Users\kalinga\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Face Recognition

In [256]:
import urllib
import cv2
import numpy as np
from keras.models import load_model

model = load_model(r"final_model.h5")

classifier = cv2.CascadeClassifier(r'haarcascade_frontalface_default.xml')

# Enter your ip address
URL = 'http://192.168.154.241:8080/shot.jpg' 

def get_pred_label(pred):
    labels = le.inverse_transform([0,1,2,3,4])
    return f"{pred} --> {labels[pred]}"

def preprocess(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img,(100,100))
    img = cv2.equalizeHist(img)
    img = img.reshape(1,100,100,1)
    img = img/255
    return img

ret = True
while ret:
    
    img_url = urllib.request.urlopen(URL)
    image = np.array(bytearray(img_url.read()),np.uint8)
    frame = cv2.imdecode(image,-1)
    
    faces = classifier.detectMultiScale(frame,1.5,5)
      
    for x,y,w,h in faces:
        face = frame[y:y+h,x:x+w]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),5)
        cv2.putText(
            frame,get_pred_label(
                np.argmax(model2.predict(preprocess(face)))
                ),
                (200,200),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2
            )
        
    cv2.imshow("capture",frame)
    if cv2.waitKey(1)==ord('q'):
        break

cv2.destroyAllWindows()



1/1 [==============================] - 0s 28ms/step
